In [15]:
import sys
import os
import pandas as pd
import nltk
import string
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import re
import random

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer


In [16]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_directory)
print("Updated sys.path:", sys.path)

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../utils')))

Updated sys.path: ['/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/ittichaiboonyarakthunya/Library/Caches/pypoetry/virtualenvs/is-project-thailand-tourism-recommendation-qwL-LsgH-py3.12/lib/python3.12/site-packages', '/Users/ittichaiboonyarakthunya/Documents/WorkDir/developments/utils', '/Users/ittichaiboonyarakthunya/Documents/WorkDir/developments/IS_Project_Thailand_Tourism_Recommendation/utils', '/Users/ittichaiboonyarakthunya/Documents/WorkDir/developments/IS_Project_Thailand_Tourism_Recommendation', '/Users/ittichaiboonyarakthunya/Documents/WorkDir/developments/IS_Project_Thailand_Tourism_Recommendation/utils', '/Users/ittichaiboonyarakthunya/Documents/WorkDir/developments/IS_Project_Thailand_Tourism_Recomm

In [17]:
from utils.data_preparation import DataPreparation

DP = DataPreparation()
DP.check_nltk_path()

True


In [18]:
FINE_TUNED_DIR = '../pretained_or_finetune-models'
REVIEWS_DATASET_DIR = '../dataset'
UTILS_DIR = '../utils'
NLTK_DATA_PATH = f"{FINE_TUNED_DIR}/nltk_data"

nltk.data.path.append(NLTK_DATA_PATH)

custom_stopwords = set(stopwords.words('english'))
stop_words = custom_stopwords
lemmatizer = WordNetLemmatizer()

In [19]:
tourism_reviews_en = pd.read_csv(f"{REVIEWS_DATASET_DIR}/eng_reviews_emotion_classify.csv", encoding='utf-8')

columns_to_use = ['helpful_votes', 'location_id', 'review_id', 'review',
                  'review_subject', 'trip_type', 'rating',
                  'location_name', 'province', 'place_id', 'emotion', 'cleaned_review']
columns_to_train = ['location_id', 'review', 'rating']
review_df = tourism_reviews_en[columns_to_use]

In [20]:
tourism_reviews_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5909 entries, 0 to 5908
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0.1    5909 non-null   int64 
 1   Unnamed: 0      5909 non-null   int64 
 2   helpful_votes   5909 non-null   int64 
 3   location_id     5909 non-null   int64 
 4   lang            5909 non-null   object
 5   review_id       5909 non-null   int64 
 6   review          5909 non-null   object
 7   review_subject  5909 non-null   object
 8   trip_type       5892 non-null   object
 9   travel_date     5892 non-null   object
 10  rating          5909 non-null   int64 
 11  location_name   5909 non-null   object
 12  province        5909 non-null   object
 13  place_id        5909 non-null   object
 14  cleaned_review  5909 non-null   object
 15  emotion         5909 non-null   object
dtypes: int64(6), object(10)
memory usage: 738.8+ KB


In [21]:
review_df.describe()

,helpful_votes,location_id,review_id,rating
count,5909.000000,5.909000e+03,5.909000e+03,5909.000000
mean,1.058216,6.483167e+06,7.590446e+08,4.153833
std,5.625108,4.877561e+06,1.844114e+08,1.067077
min,0.000000,3.110380e+05,1.049076e+08,1.000000
25%,0.000000,2.578107e+06,6.573250e+08,4.000000
50%,0.000000,6.017599e+06,7.832754e+08,4.000000
75%,1.000000,8.818088e+06,9.240324e+08,5.000000
max,182.000000,2.714283e+07,9.653184e+08,5.000000


In [22]:
# df_review_yelp = pd.read_csv("../../dataset/Central_Attraction_Reviews.csv")
# df_review_yelp.head(5)


# custom_review_df = df_review_yelp[["Rating", "Review Summary"]]
# new_df_renamed = custom_review_df.rename(columns={"Rating": "label", "Review Summary": "text"})
# df_review_yelp = new_df_renamed
# df_review_yelp.head(5)

# df_review_yelp['location_id'] = df_review_yelp['text'].apply(lambda x: map_generate_location_id())
# df_review_yelp['review'] = df_review_yelp['text']
# df_review_yelp['rating'] = df_review_yelp['label'].apply(map_label_to_rating)

# df_review_yelp = df_review_yelp[['rating', 'review', 'location_id']]

# df_postive_yelp = df_review_yelp[df_review_yelp['rating'].isin([5])]
# df_natural_yelp = df_review_yelp[df_review_yelp['rating'].isin([3, 4])]
# df_negative_yelp = df_review_yelp[df_review_yelp['rating'].isin([1, 2])]

# print(f"Positive [{len(df_postive_yelp)}] / Natural [{len(df_natural_yelp)}] / Negative [{len(df_negative_yelp)}]")

# combined_positive_df = pd.concat([df_postive_yelp.iloc[:-len(df_postive)], df_postive])
# combined_natural_df = pd.concat([df_natural_yelp.iloc[:-len(df_natural)], df_natural])
# combined_negative_df = pd.concat([df_negative_yelp.iloc[:-len(df_negative)], df_negative])

# selected_column_trained = ['rating', 'location_id', 'review']
# full_combined_df = pd.concat([df_natural[selected_column_trained], df_postive[selected_column_trained], df_negative[selected_column_trained]])

In [23]:
def map_label_to_rating(val):
    return val + 1

def map_generate_location_id():
    return f"CUSTOM{random.randint(10000, 99999)}"


In [24]:
# df_postive = review_df[review_df['rating'].isin([5])]
# df_natural = review_df[review_df['rating'].isin([3, 4])]
# df_negative = review_df[review_df['rating'].isin([1, 2])]

# print(f"Positive [{len(df_postive)}] / Natural [{len(df_natural)}] / Negative [{len(df_negative)}]")

selected_column_trained = ['rating', 'location_id', 'review']
# full_combined_df = pd.concat([df_natural[selected_column_trained], df_postive[selected_column_trained], df_negative[selected_column_trained]])

full_combined_df = review_df[selected_column_trained]

In [25]:


full_combined_df['cleaned_review'] = full_combined_df['review'].apply(DP.clean_text)

# 2. สร้าง Word Embeddings ด้วย Word2Vec
# word2vec_model = Word2Vec(sentences=review_df['cleaned_review'], vector_size=300, window=5, min_count=1, sg=1)  # ใช้ Skip-Gram
sentences = [row.split() for row in full_combined_df['cleaned_review']]
word2vec_model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4) 

/var/folders/fk/449tdf_12hzd2r7ps4659q0m0000gn/T/ipykernel_40155/792440960.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_combined_df['cleaned_review'] = full_combined_df['review'].apply(DP.clean_text)


In [26]:
# Parameters
embedding_dim = 300  # Assuming Word2Vec has 300 dimensions
gru_units = 128
dropout_rate = 0.5
num_classes = 1  # Binary classification
max_sequence_length = 300  # Maximum length of input sequences
ngram_range = (1, 2)  # Unigrams and bigrams
max_features = 20000  

# Create the embedding matrix from the Word2Vec model
word_index = {word: i for i, word in enumerate(word2vec_model.wv.index_to_key)}
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

In [27]:
def text_to_word2vec(tokens, model, max_length=300):
    vector = np.zeros((max_length, 300))
    for i, word in enumerate(tokens):
        if i < max_length:
            if word in model.wv:
                vector[i] = model.wv[word]
    return vector

def map_rating_to_sentiment(rating):
    if rating in [1, 2]:
        return 'negative'
    elif rating in [3]:
        return 'neutral'
    else:  # 4 or 5
        return 'positive'

In [28]:
full_combined_df['embedding'] = review_df['cleaned_review'].apply(lambda x: text_to_word2vec(x, word2vec_model))
full_combined_df['sentiment'] = full_combined_df['rating'].apply(map_rating_to_sentiment)

/var/folders/fk/449tdf_12hzd2r7ps4659q0m0000gn/T/ipykernel_40155/3550278844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_combined_df['embedding'] = review_df['cleaned_review'].apply(lambda x: text_to_word2vec(x, word2vec_model))
/var/folders/fk/449tdf_12hzd2r7ps4659q0m0000gn/T/ipykernel_40155/3550278844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_combined_df['sentiment'] = full_combined_df['rating'].apply(map_rating_to_sentiment)


In [43]:
from sklearn.utils import resample

# Assuming `review_df` is your original DataFrame with a 'sentiment' column

# 1. Separate the data by class
df_positive = full_combined_df[full_combined_df.sentiment == 'positive']
df_neutral = full_combined_df[full_combined_df.sentiment == 'neutral']
df_negative = full_combined_df[full_combined_df.sentiment == 'negative']

# 2. Resample the minority classes
# For example, oversample the neutral and negative classes to match the positive class
df_neutral_upsampled = resample(df_neutral,
                                replace=True,  # Sample with replacement
                                n_samples=len(df_positive),  # Match number of positive class samples
                                random_state=42)  # Reproducible results

df_negative_upsampled = resample(df_negative,
                                 replace=True,  # Sample with replacement
                                 n_samples=len(df_positive),  # Match number of positive class samples
                                 random_state=42)  # Reproducible results

# 3. Combine the resampled DataFrames
df_balanced = pd.concat([df_positive, df_neutral_upsampled, df_negative_upsampled])

# 4. Shuffle the combined DataFrame
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)




In [44]:
df_balanced

,rating,location_id,review,cleaned_review,embedding,sentiment
0,3,4322585,This was an ornate wat inside Mae Sot. It is a...,ornate wat inside mae sot burmese style wat ty...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",neutral
1,2,2667297,Got the Asia perfect for body treatment. We re...,got asia perfect body treatment read review th...,"[[0.000530912249814719, 0.150453582406044, 0.0...",negative
2,4,10388576,A few years ago a huge market was built in Ban...,year ago huge market built ban phe totally dis...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",positive
3,1,6885224,The visit to this island is worth it just to u...,visit island worth understand tragic result un...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",negative
4,5,6669325,So many different bananas! And a chance to see...,many different banana chance see stored taste ...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",positive
...,...,...,...,...,...,...
14062,3,7287889,It is not as pretty as some picture lead to be...,pretty picture lead believe big either minute ...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",neutral
14063,1,10631803,The whole trip was spoilt by lying local tour ...,whole trip spoilt lying local tour package sel...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",negative
14064,3,1368429,"As a keen biker, I was excited to experience t...",keen biker excited experience breathtaking sce...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",neutral
14065,5,8300428,This place is really beautiful and was decorat...,place really beautiful decorated statue mae na...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",positive


In [45]:
print("Number of rows in dataframe before saving:", df_balanced.shape)


Number of rows in dataframe before saving: (14067, 6)


In [46]:
print(df_balanced['sentiment'].value_counts())

sentiment
neutral     4689
negative    4689
positive    4689
Name: count, dtype: int64


In [47]:
df_balanced.to_csv("dataset/embled_review_combined.csv", index=False, na_rep='N/A', header=True, sep=',', encoding='utf-8')